PinSage Graph Recommendation using ogbn.
Jay Urbain, PhD   
5/30/2023

References:   

Building product graphs automatially, Amazon.     
https://www.amazon.science/blog/building-product-graphs-automatically

Amazon Product co-purchasing networks dataset:    
http://snap.stanford.edu/data/index.html#amazon  

GraphSage:    
https://arxiv.org/abs/1706.02216  

ogbn-products:
- Undirecrted, unweighted graph representing product co-purchasing network to
predict shopping preferences.    
https://ogb.stanford.edu/docs/nodeprop/#ogbn-products

Open Benchmark Dataset:   
https://ogb.stanford.edu/  

Blog articles:   

https://medium.com/pinterest-engineering/pinsage-a-new-graph-convolutional-neural-network-for-web-scale-recommender-systems-88795a107f48

https://towardsdatascience.com/a-comprehensive-case-study-of-graphsage-algorithm-with-hands-on-experience-using-pytorchgeometric-6fc631ab1067.   



In [1]:
# !nvidia-smi

# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-2.6.0+cu124.html
  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp311-cp311-linux_x86_64.whl size=3620314 sha256=52ef4de154ab72c29eeef6f64391f504d898f9553d68896aae626bca8302b9a5
  Stored in directory: /root/.cache/pip/wheels/b8/d4/0e/a80af2465354ea7355a2c153b11af2da739cfcf08b6c0b28e2
Successfully built torch-scatter
Looking in links: https://pytorch-geometric.com/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp311-cp311-linux_x86_64.whl size=2813759 sha256=093c5bf95196ea7a5729b6de29c29e6f630826e40caaade94fffe0d8af88a914
  Stored in directory: /root/.cache/pip/wheels/75/e2/1e/299c596063839303657c211f587f05591891cc6cf126d94d21
Successfully built torch-sparse


In [2]:
!pip install wandb

In [3]:
!pip install pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.8 MB/s eta 0:00:00


Dataset  

Network collected by crawling Amazon website.  If a product is frequently co-purchased with product j, the graph contains a directed edge from i to j.
It is based on Customers Who Bought This Item Also Bought feature of the Amazon website.
The data was collected in March 02 2003.
The graph has 262,111 nodes and 1,234,877 edges. Basic format of the data:

```
# Directed graph (each unordered pair of nodes is saved once): Amazon0302.txt
# Amazon product co-purchaisng network from March 02 2003
# Nodes: 262111 Edges: 1234877
# FromNodeId	ToNodeId
0	1
0	2
0	3
0	4
0	5
1	0
```


## Building the Graph


Treat each product as a node. Have an edge between any pair of products that are frequently bought together. Called a homogenous graph i.e. a graph where each node represents one type of entity and each edge represents one type of relationship.

Can make this more complex by adding more information. For example: add users as nodes to the graph and add edges between users and the products that they have bought. This would make the graph heterogenous i.e. a graph which can have nodes of multiple types and similarly edges of multiple types which can represent a variety of relationships between the nodes.


### Loading the data into a PyTorch Geometric Graph

Need to convert data into a format which can be processed easily by PyG.

Read all the lines in the file, initialize a numpy array and a list to keep track of the in-degree of each node and the edges respectively.

Then all the lines are read one by one and processed: the lines with metadata are ignored and the lines with the start node and end node are processed. The in-degree of the end node is incremented and the edge data is added to edge_index.

Use the in-degree of each node and the edge_index to create a PyG graph using the Data class.

The graph is then saved using torch.save and optionally uploaded as a W&B artifact


In [4]:
import numpy as np
import torch
from torch_geometric.data import Data
import wandb

wandb.init(project="gnn-recommender", job_type="preprocessing", save_code=True)
file_path = wandb.use_artifact("manan-goel/gnn-recommender/raw-data:v0").download()
with open(f'{file_path}/amazon0302.txt', 'r') as f:
    edges = f.readlines()

    idx = 0
    edge_index = []
    in_degrees = np.zeros((262111, 1))

    while idx < len(edges):
        print(f"{idx}/{len(edges)}", end='\r')
        line = edges[idx]
        if line.startswith('#'):
            idx += 1
            continue
        start, end = line.strip().split()
        start, end = int(start), int(end)
        in_degrees[end][0] += 1

        edge_index.append([start, end])
        idx += 1

    edge_index = torch.tensor(edge_index).t().contiguous()
    graph = Data(x=in_degrees, edge_index=edge_index)

    graph_artifact = wandb.Artifact('amazon_product_graph', type='graph')

    torch.save(graph, 'amazon0302.pt')

    graph_artifact.add_file('amazon0302.pt')
    wandb.log_artifact(graph_artifact)
wandb.finish()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jayurbain to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


### Visualizing the PyTorch Geometric Graph

Hard to visualize hundreds of thousands of nodes so sample the first 100 nodes from the graph using the subgraph utility from PyTorch Geometric.

Use the metadata of the nodes available as a part of the dataset for creating a more informative visualization.

This is very slow.


In [ ]:
import pickle

import numpy as np
import torch
from pyvis.network import Network
from torch_geometric import utils
from torch_geometric.data import Data
from tqdm.auto import tqdm

import wandb


# Download the parsed metadata and smaller graph
wandb.init(project="gnn-recommender", job_type="eda", save_code=True)
metadata_path = wandb.use_artifact("manan-goel/gnn-recommender/processed_metadata:latest").download()
graph_path = wandb.use_artifact("manan-goel/gnn-recommender/smaller_graph:latest").download()

g = torch.load(f'{graph_path}/smaller_graph.pt', weights_only=False)

with open(f'{metadata_path}/metadata.pkl', 'rb') as f:
    metadata = pickle.load(f)

# Initialize the PyVis network
net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")

# Add the edges from the PyG graph to the PyVis network
for e in tqdm(g.edge_index.T):

    src = e[0].item()
    dst = e[1].item()
    if src == 0 or dst == 0:
        continue
    src_title = "Title:" + metadata[src]['title'] + "\n\n" + "Categories:\n" + "\n".join(list(metadata[src]['categories'])[:3])
    dst_title = "Title:" + metadata[dst]['title'] + "\n\n" + "Categories:\n" + "\n".join(list(metadata[dst]['categories'])[:3])
    net.add_node(dst, label=src_title, title=src_title)
    net.add_node(src, label=dst_title, title=dst_title)
    net.add_edge(src, dst, value=0.1)

# Save the PyVis visualisation to a HTML file
net.show("graph.html")

# Log the interactive HTML to W&B
wandb.log({"eda/graph": wandb.Html("graph.html")})

wandb.finish()

wandb: Downloading large artifact processed_metadata:latest, 71.56MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.5
wandb:   1 of 1 files downloaded.  


  0%|          | 0/218424 [00:00<?, ?it/s]

### Creating Node Features

For every node in the graph, provide the model with some information about what the node represents.

A basic way of doing this is using the in-degree of the node as a feature. But using in-degree does not add any information about what the product represents so it may not lead to good performance.

So it's important to "featurize" the nodes in the most robust way possible. For example, use Doc2Vec embeddings for the product titles of each node and use the embedding as the input node feature.

Another possible option is to use metadata or a list of categories to which each product belongs.

### The Link Prediction Problem

One of the fundamental problems in geometric deep learning is link prediction, i.e. predicting whether an edge exists between two nodes in a graph.

This can be used to predict whether two users can be friends in a social network,  predicting interactions between genes and proteins in a biological network, or how similar or related two products are.

Basic idea is to see if two products are similar and hence, can be recommended as suggestions when someone is looking at one of them or not.

Basic way to measure similarity is using graph embeddings. Graph embedding algorithms learn an embedding space in which neighboring nodes are represented by vectors so that vector similarity measures, such as dot product similarity, or euclidean distance, hold in the embedding space.

These can be learned using graph convolutional neural networks (GCNs).


### Link Prediction

#### Splitting the Dataset

Create a train, test and validation split of the edges in the dataset.



In [7]:
import torch
from torch_geometric.transforms import RandomLinkSplit
import wandb


# Download and load the graph from W&B artifacts
wandb.init(project="gnn-recommender", job_type="preprocessing", save_code=True)
wandb.use_artifact("manan-goel/gnn-recommender/smaller_graph:latest")
graph = torch.load('smaller_graph.pt')


# Add 5000 edges in the validation and test sets respectively
transform = RandomLinkSplit(num_val=5000, num_test=5000, is_undirected=True, split_labels=True)
train_data, val_data, test_data = transform(graph)


# Save the splits and save as W&B artifacts
torch.save(train_data, 'train.pt')
torch.save(val_data, 'val.pt')
torch.save(test_data, 'test.pt')


artifact = wandb.Artifact('split_smaller_graph', type='graph')
artifact.add_file('train.pt')
artifact.add_file('val.pt')
artifact.add_file('test.pt')


wandb.log_artifact(artifact)
wandb.finish()


FileNotFoundError: [Errno 2] No such file or directory: 'smaller_graph.pt'

Implementing the Model consists of two parts: arriving at a node embedding using graph convolutions for two nodes followed by using the two embeddings to make a final prediction whether a link exists or not.

Graph Convolution
There are multiple kinds of graph convolution models available in PyTorch Geometric. Using GraphSAGE model.


In [ ]:
import torch_geometric as pyg
import torch
from torch import nn
import torch.nn.functional as F


class GNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout):
        super(GNNStack, self).__init__()
        conv_model = pyg.nn.SAGEConv


        self.convs = nn.ModuleList()
        self.convs.append(conv_model(input_dim, hidden_dim))
        self.dropout = dropout
        self.num_layers = num_layers


        # Create num_layers GraphSAGE convs
        assert (self.num_layers >= 1), 'Number of layers is not >=1'
        for l in range(self.num_layers - 1):
            self.convs.append(conv_model(hidden_dim, hidden_dim))


        # post-message-passing processing
        self.post_mp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim), nn.Dropout(self.dropout),
            nn.Linear(hidden_dim, output_dim))


    def forward(self, x, edge_index):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)


        x = self.post_mp(x)


        # Return final layer of embeddings if specified
        return x

Link Prediction

The previous module provides an embedding for a pair of nodes. This module is responsible for combining the two embeddings and making a binary prediction.

This module is extremely similar to a full connected neural network with multiple linear layers stacked one after another.

Forward Pass

Take the embeddings for the two nodes as arguments and performs an element-wise product of the two. The obtained feature is then propagated through the linear layers and finally the sigmoid activation function is applied to find the final probability of their being an edge between the two nodes or not.

In [ ]:
class LinkPredictor(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(LinkPredictor, self).__init__()

        # Create linear layers
        self.lins = nn.ModuleList()
        self.lins.append(nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x_i, x_j):
        # x_i and x_j are both of shape (E, D)
        x = x_i * x_j
        for lin in self.lins[:-1]:
            x = lin(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.sigmoid(x)

Training the Model

Training a link prediction model brings up a very interesting problem: the dataset we possess is a list of edges in the graph and when you think about it as a binary classification problem, this means we only have positive samples. Hence, there exists a concept called 'negative edges' i.e. edges that do not actually exist in the graph which we consider as negative samples. PyTorch Geometric provides a utility for this as well.


In [ ]:
def train(model, link_predictor, emb, edge_index, pos_train_edge, batch_size, optimizer):
    model.train()
    link_predictor.train()

    train_losses = []

    for edge_id in tqdm(pyg.loader.DataLoader(range(pos_train_edge.shape[0]), batch_size, shuffle=True), leave=True):
        optimizer.zero_grad()

        # Run message passing on the inital node embeddings to get updated embeddings
        node_emb = model(emb, edge_index)  # (N, d)

        # Predict the class probabilities on the batch of positive edges using link_predictor
        pos_edge = pos_train_edge[edge_id].T  # (2, B)
        pos_pred = link_predictor(node_emb[pos_edge[0]], node_emb[pos_edge[1]])  # (B, )

        # Sample negative edges (same number as number of positive edges) and predict class probabilities
        neg_edge = negative_sampling(edge_index, num_nodes=emb.shape[0],
                                     num_neg_samples=edge_id.shape[0], method='dense')  # (Ne,2)
        neg_pred = link_predictor(node_emb[neg_edge[0]], node_emb[neg_edge[1]])  # (Ne,)

        # Compute the corresponding negative log likelihood loss on the positive and negative edges
        loss = -torch.log(pos_pred + 1e-15).mean() - torch.log(1 - neg_pred + 1e-15).mean()

        # Backpropagate and update parameters
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
    return sum(train_losses) / len(train_losses)

In [ ]:
To finally initialize all the modules and train the model, you can use the following snippet

In [ ]:
train_graph = torch.load('train.pt')
val_graph = torch.load('val.pt')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optim_wd = 0
epochs = 300
hidden_dim = 1024
dropout = 0.3
num_layers = 2
lr = 1e-5
node_emb_dim = 1
batch_size = 1024


train_graph = train_graph.to(device)
val_graph = val_graph.to(device)




model = GNNStack(node_emb_dim, hidden_dim, hidden_dim, num_layers, dropout, emb=True).to(device) # the graph neural network that takes all the node embeddings as inputs to message pass and agregate
link_predictor = LinkPredictor(hidden_dim, hidden_dim, 1, num_layers + 1, dropout).to(device)


optimizer = torch.optim.Adam(
    list(model.parameters()) + list(link_predictor.parameters()),
    lr=lr, weight_decay=optim_wd
)


train_loss = train(
	model,
	link_predictor,
	torch.tensor(train_graph.x).float().to(device),
	train_graph.edge_index,
	train_graph.pos_edge_label_index.T,
	batch_size,
	optimizer
)




Validating Model Performance
One of the best metrics to validate model performance is measuring Hits@K which is the count of how many positive triples are ranked in the top-n positions against a bunch of synthetic negativ


Testing
For testing the performance of the model, we took 5,000 nodes from the product graph that had not been seen so far.  Initially we took 50,000 nodes from the product graph and for testing we took nodes that were not a part of this set of nodes. To test it, we obtained the learned embeddings of all the nodes in the test set and for 10 of them we saw which other nodes they were connected to using our model.
